In [1]:
import requests
from bs4 import BeautifulSoup

urls = ["https://store.steampowered.com/search/?filter=topsellers&tags=597&supportedlang=english&ndl=1"]

#Action "https://store.steampowered.com/search/?tags=19&supportedlang=english&filter=topsellers&ndl=1" 
#SinglePlayer https://store.steampowered.com/search/?filter=topsellers&tags=4182&supportedlang=english&ndl=1
#Adventure https://store.steampowered.com/search/?filter=topsellers&tags=21&supportedlang=english&ndl=1
#Indie https://store.steampowered.com/search/?filter=topsellers&tags=492&supportedlang=english&ndl=1
#Simulation https://store.steampowered.com/search/?filter=topsellers&tags=599&supportedlang=english&ndl=1
#Strategy https://store.steampowered.com/search/?filter=topsellers&tags=9&supportedlang=english&ndl=1
#Casual https://store.steampowered.com/search/?filter=topsellers&tags=597&supportedlang=english&ndl=1
#RPG https://store.steampowered.com/search/?filter=topsellers&tags=122&supportedlang=english&ndl=1
#Multiplayer https://store.steampowered.com/search/?filter=topsellers&tags=3859&supportedlang=english&ndl=1
#Atmospheric https://store.steampowered.com/search/?filter=topsellers&tags=4166&supportedlang=english&ndl=1
#2d https://store.steampowered.com/search/?filter=topsellers&tags=3871&supportedlang=english&ndl=1 
#story rich https://store.steampowered.com/search/?filter=topsellers&tags=1742&supportedlang=english&ndl=1
#co-op https://store.steampowered.com/search/?filter=topsellers&tags=1685&supportedlang=english&ndl=1
#first-person https://store.steampowered.com/search/?filter=topsellers&tags=3839&supportedlang=english&ndl=1
#3d https://store.steampowered.com/search/?filter=topsellers&tags=4191&supportedlang=english&ndl=1
#open world https://store.steampowered.com/search/?filter=topsellers&tags=1695&supportedlang=english&ndl=1

app_id = []

for url in urls:
    response = requests.get(url)
# , headers={
#     'cookie': 'steamCountry=KR!13817a80e36b14f1a94dd77fb2142a64; browserid=3999848730290483524; sessionid=2225f1149bbcf81b9631038f; timezoneOffset=32400,0; app_impressions=2344520@1_7_7_7000_150_1|2138330@1_7_7_7000_150_1|2358720@1_7_7_7000_150_1|2074920@1_7_7_7000_150_1|2183900@1_7_7_7000_150_1|1245620@1_7_7_7000_150_1|2139460@1_7_7_7000_150_1|1049590@1_7_7_7000_150_1|1091500@1_7_7_7000_150_1|578080@1_7_7_7000_150_1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
#     'accept-language':'ko-KR,ko;q=0.9',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'accept-encoding': 'gzip, deflate, br, zstd',
#     'cache-control':'no-cache'
# }

    soup = BeautifulSoup(response.content, 'html.parser')

    app_ids = []

    container = soup.find('div', id='search_result_container')

    if container:
        for item in container.find_all('a'):
            link =item['href']
            if 'app' in link:
                ex = link.split('app')[1].split('/')[1]
                app_ids.append(ex)

    app_ids=app_ids[:10] # 상위 10개

    for i in range(len(app_ids)):
        app_id.append(app_ids[i])

app_id


['2198150',
 '985810',
 '1260320',
 '2739990',
 '960170',
 '1621690',
 '1222670',
 '787480',
 '2357570',
 '3155240']

In [2]:
import requests
import pandas as pd

# Steam Store API에서 특정 게임의 세부 정보를 가져오는 함수
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]["success"]:
            game_data = data[str(app_id)]["data"]
            return {
                "app_id": app_id,
                "name": game_data.get("name"),
                "genres": ", ".join([genre['description'] for genre in game_data.get("genres", [])]),  # 장르
                "header_image": game_data.get("header_image"),  # 헤더 이미지
                "release_date": game_data.get("release_date", {}).get("date"),  # 출시일
                "price": game_data.get("price_overview", {}).get("final_formatted", "가격 정보 없음")  # 가격 정보
            }
    return {}

# Steam API를 통해 리뷰 데이터를 가져오는 함수
def get_reviews_for_game(app_id, num_reviews=100):
    url = f"https://store.steampowered.com/appreviews/{app_id}"
    params = {
        'json': 1,
        'num_per_page': num_reviews,  # 한 번에 가져올 리뷰 수
        'filter': 'recent',  # 최신 리뷰만 가져옴
        'language': 'english'  # 영어로 된 리뷰만 가져옴
    }
    response = requests.get(url, params=params)
    data = response.json()

    reviews = []

    if 'reviews' in data:
        for review in data['reviews']:
            reviews.append({
                'review_text': review.get('review'),  # 리뷰 내용
                'review_score': review.get('voted_up'),  # 추천 여부
                'language': review.get('language'),  # 리뷰 언어
                'author_steamid': review.get('author', {}).get('steamid'),  # 작성자 Steam ID
                'playtime_at_review': review.get('author', {}).get('playtime_at_review'),  # 리뷰 작성 당시의 플레이 시간
                'timestamp_created': review.get('timestamp_created')  # 리뷰 작성 시간
            })

    return reviews

# Steam ID를 사용자 닉네임으로 변환하는 함수 (API 키 사용하지 않음)
def resolve_vanity_url(steam_id):
    # API 키 없이, 닉네임을 직접 반환
    return steam_id  # 원래 Steam ID 반환

# 게임 이름, 장르, 이미지, 리뷰 정보 등을 포함한 데이터를 DataFrame으로 정리하는 함수
def get_game_data_with_reviews(app_ids, num_reviews=100):
    all_data = []

    for app_id in app_ids:
        # 1. 게임 세부 정보 가져오기
        game_info = get_game_details(app_id)
        
        if game_info:
            # 2. 게임에 대한 리뷰 가져오기
            reviews = get_reviews_for_game(app_id, num_reviews)
            
            # 3. 각 리뷰에 게임 세부 정보 추가
            for review in reviews:
                # 4. Steam ID를 사용자 닉네임으로 변환 (닉네임으로 처리됨)
                author_nickname = resolve_vanity_url(review['author_steamid'])
                
                all_data.append({
                    'app_id': game_info['app_id'],  # 게임 ID
                    'game_name': game_info['name'],  # 게임 이름
                    'game_genres': game_info['genres'],  # 게임 장르
                    'game_image': game_info['header_image'],  # 게임 이미지
                    'game_release_date': game_info['release_date'],  # 출시일
                    'game_price': game_info['price'],  # 가격
                    'review_text': review['review_text'],  # 리뷰 내용
                    'review_score': review['review_score'],  # 리뷰 평가
                    'language': review['language'],  # 리뷰 언어
                    'author': author_nickname,  # 작성자 닉네임
                    'playtime_at_review': review['playtime_at_review'],  # 작성 당시 플레이 시간
                    'timestamp_created': review['timestamp_created']  # 리뷰 작성 시간
                })

    # 모든 데이터를 DataFrame으로 변환
    df = pd.DataFrame(all_data)
    return df

# 게임 데이터 및 리뷰 가져오기
df_game_data = get_game_data_with_reviews(app_id, num_reviews=50)  # 각 게임당 50개의 리뷰 가져오기

# CSV 파일로 저장
df_game_data.to_csv('Casual.csv', index=False)


In [3]:
Example=pd.read_csv('Casual.csv')
Example

,app_id,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020","Once Tiny glade dropped as a free demo, I trie...",True,english,76561198038828872,185.0,1727244528
1,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",This deserves GOTY for sure. Just Therapeutic ♡,True,english,76561198219808051,608.0,1727244216
2,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",beautiful game,True,english,76561198016025460,206.0,1727243929
3,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",This game is so good and satisfying. I highly ...,True,english,76561198985925356,286.0,1727243851
4,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020","Medevil-fantasy vibes, The Game",True,english,76561199482488960,318.0,1727243723
...,...,...,...,...,...,...,...,...,...,...,...,...
362,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",Never stopped playing this game when I got it....,True,english,76561198079702081,NaN,1724909058
363,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",Good,True,english,76561198356910715,NaN,1724905629
364,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",LENA CHAN UUUUUUUOOOOOOOOOHHHHHHHHHH,True,english,76561198198292809,NaN,1724904791
365,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",DJMAX BOOBA UPDATE YES YES YES YES YES YES YES,True,english,76561197997052272,NaN,1724903183


In [4]:
Example['app_id'].value_counts()

app_id
2198150    50
1260320    50
960170     50
1621690    50
1222670    50
787480     50
2357570    50
3155240    17
Name: count, dtype: int64

In [5]:
Example1= Example.copy()
Example1

,app_id,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020","Once Tiny glade dropped as a free demo, I trie...",True,english,76561198038828872,185.0,1727244528
1,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",This deserves GOTY for sure. Just Therapeutic ♡,True,english,76561198219808051,608.0,1727244216
2,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",beautiful game,True,english,76561198016025460,206.0,1727243929
3,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",This game is so good and satisfying. I highly ...,True,english,76561198985925356,286.0,1727243851
4,2198150,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020","Medevil-fantasy vibes, The Game",True,english,76561199482488960,318.0,1727243723
...,...,...,...,...,...,...,...,...,...,...,...,...
362,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",Never stopped playing this game when I got it....,True,english,76561198079702081,NaN,1724909058
363,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",Good,True,english,76561198356910715,NaN,1724905629
364,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",LENA CHAN UUUUUUUOOOOOOOOOHHHHHHHHHH,True,english,76561198198292809,NaN,1724904791
365,3155240,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",DJMAX BOOBA UPDATE YES YES YES YES YES YES YES,True,english,76561197997052272,NaN,1724903183


In [6]:
Example1=Example1.drop(['app_id', ], axis=1) # app_id 삭제
Example1 = Example1[Example1['language'] == 'english'] # english 
Example1 = Example1.reset_index(drop=True) # 재정렬
Example1 

,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020","Once Tiny glade dropped as a free demo, I trie...",True,english,76561198038828872,185.0,1727244528
1,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",This deserves GOTY for sure. Just Therapeutic ♡,True,english,76561198219808051,608.0,1727244216
2,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",beautiful game,True,english,76561198016025460,206.0,1727243929
3,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020",This game is so good and satisfying. I highly ...,True,english,76561198985925356,286.0,1727243851
4,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,"23 Sep, 2024","₩ 14,020","Medevil-fantasy vibes, The Game",True,english,76561199482488960,318.0,1727243723
...,...,...,...,...,...,...,...,...,...,...,...
362,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",Never stopped playing this game when I got it....,True,english,76561198079702081,NaN,1724909058
363,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",Good,True,english,76561198356910715,NaN,1724905629
364,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",LENA CHAN UUUUUUUOOOOOOOOOHHHHHHHHHH,True,english,76561198198292809,NaN,1724904791
365,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,"28 Aug, 2024","₩ 14,800",DJMAX BOOBA UPDATE YES YES YES YES YES YES YES,True,english,76561197997052272,NaN,1724903183


In [7]:
# Example1 전처리
Example1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   game_name           367 non-null    object 
 1   game_genres         367 non-null    object 
 2   game_image          367 non-null    object 
 3   game_release_date   367 non-null    object 
 4   game_price          367 non-null    object 
 5   review_text         367 non-null    object 
 6   review_score        367 non-null    bool   
 7   language            367 non-null    object 
 8   author              367 non-null    int64  
 9   playtime_at_review  350 non-null    float64
 10  timestamp_created   367 non-null    int64  
dtypes: bool(1), float64(1), int64(2), object(7)
memory usage: 29.2+ KB


In [8]:
Example1['game_name'].isnull()
Example1['game_name'].value_counts() # 개수확인  총합 .sum()
Example1['game_name'].isnull() # null 확인 총합 .sum()

0      False
1      False
2      False
3      False
4      False
       ...  
362    False
363    False
364    False
365    False
366    False
Name: game_name, Length: 367, dtype: bool

In [9]:
Example1['game_genres'].value_counts().sum()
Example1['game_genres'].isnull().sum()

0

In [10]:
Example1['game_image'].value_counts().sum()
Example1['game_image'].isnull().sum()



0

In [11]:
Example1['language'].value_counts().sum()
Example1['language'].isnull().sum()

0

In [12]:
Example1['game_release_date'].value_counts().sum()
Example1['game_release_date'].isnull().sum()
Example1['game_release_date'].value_counts()


# 이탈리아어 월 이름을 영어로 변환하는 사전
month_map = {
    'gen.': 'Jan',
    'feb.': 'Feb',
    'mar.': 'Mar',
    'apr.': 'Apr',
    'mag.': 'May',
    'giu.': 'Jun',
    'lug.': 'Jul',
    'ago.': 'Aug',
    'set.': 'Sep',
    'ott.': 'Oct',
    'nov.': 'Nov',
    'dic.': 'Dec'
}

# 날짜 변환 함수 정의
def parse_date(date_str):
    # 이탈리아어 월 이름을 영어로 변환
    for italian_month, english_month in month_map.items():
        if italian_month in date_str:
            date_str = date_str.replace(italian_month, english_month)

    # 여러 가지 날짜 형식 처리
    try:
        # 'Sep 19, 2024' 형식
        return pd.to_datetime(date_str, format='%b %d, %Y')
    except ValueError:
        pass
    try:
        # '19 Sep, 2024' 형식
        return pd.to_datetime(date_str, format='%d %b, %Y')
    except ValueError:
        pass
    try:
        # '9/set./2024' 형식 (이탈리아어 월 변환 후 처리)
        return pd.to_datetime(date_str, format='%d/%b/%Y')
    except ValueError:
        pass
    
    return pd.NaT  # 변환 실패 시 NaT 반환

# 데이터프레임의 날짜 열 변환
Example1['game_release_date'] = Example1['game_release_date'].apply(parse_date)
Example1['game_release_date'].isnull().sum()
Example1['game_release_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 367 entries, 0 to 366
Series name: game_release_date
Non-Null Count  Dtype         
--------------  -----         
367 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.0 KB


In [13]:
Example1['review_score']
Example1['review_score'].isnull().sum()
Example1['review_score'].value_counts()
Example1['review_score'].value_counts().sum()

367

In [14]:
Example1['review_text'].value_counts().sum()
Example1['review_text'].value_counts()
Example1['review_text'].isnull().sum()
Example1['review_text'].isnull()
Example1 = Example1.dropna(subset=['review_text'], axis=0)

In [15]:
# game_price
Example1['game_price'].value_counts().sum()

367

In [16]:
Example1['game_price'].isnull().sum()


0

In [17]:
Example1['game_price'].value_counts()

game_price
가격 정보 없음    100
₩ 14,020     50
₡5.250       50
₩ 49,800     50
£15.99       50
₩ 11,480     50
₩ 14,800     17
Name: count, dtype: int64

In [18]:
# 특정 값 변경
Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') # 

Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')

In [19]:
# 특정 값 변경
Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') 

Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')

In [20]:
import re

# 통화 기호에 따라 숫자를 변환하는 함수
def extract_numbers(price_str):
    if isinstance(price_str, str):  # price_str이 문자열인지 확인
        # 통화 기호에 따라 환율을 설정
        if '₩' in price_str:
            multiplier = 1  # 원화: 그대로 사용
        elif 'CDN$' in price_str:
            multiplier = 950  # 캐나다 달러 기준, 예: 1 CAD = 950 KRW
        elif '$' in price_str:
            multiplier = 1300  # 미국 달러 기준, 예: 1 USD = 1300 KRW
        elif '€' in price_str:
            multiplier = 1400  # 유로 기준, 예: 1 EUR = 1400 KRW
        elif '¥' in price_str:
            multiplier = 10  # 엔화 기준, 예: 1 JPY = 10 KRW
        elif 'R$' in price_str:
            multiplier = 300  # 브라질 헤알 기준, 예: 1 BRL = 300 KRW
        else:
            multiplier = 1  # 통화 기호가 없는 경우 그대로 사용

        # 소수점이 포함된 숫자만 추출 (쉼표와 공백 제거)
        numbers = re.sub(r'[^\d.]', '', price_str)
        
        # 추출한 숫자 문자열을 소수로 변환한 후 환율 적용
        return float(numbers) * multiplier if numbers else 0  # 숫자에 통화 단위 곱하기
    return 0  # 문자열이 아닐 경우 0 반환

# 'game_price' 열 변환
Example1['game_price'] = Example1['game_price'].replace('가격 정보 없음', '0')  # 가격 정보가 없는 경우 0으로 대체
Example1['game_price'] = Example1['game_price'].apply(extract_numbers)  # 각 가격 문자열에 대해 함수 적용

In [21]:
Example1['game_price'].value_counts()


game_price
0.00        100
14020.00     50
5.25         50
49800.00     50
15.99        50
11480.00     50
14800.00     17
Name: count, dtype: int64

In [22]:
Example1['game_price'].isnull().sum()


0

In [23]:
Example1['game_price'].astype(int)

0      14020
1      14020
2      14020
3      14020
4      14020
       ...  
362    14800
363    14800
364    14800
365    14800
366    14800
Name: game_price, Length: 367, dtype: int32

In [24]:
Example1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   game_name           367 non-null    object        
 1   game_genres         367 non-null    object        
 2   game_image          367 non-null    object        
 3   game_release_date   367 non-null    datetime64[ns]
 4   game_price          367 non-null    float64       
 5   review_text         367 non-null    object        
 6   review_score        367 non-null    bool          
 7   language            367 non-null    object        
 8   author              367 non-null    int64         
 9   playtime_at_review  350 non-null    float64       
 10  timestamp_created   367 non-null    int64         
dtypes: bool(1), datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 29.2+ KB


In [25]:
# review_text 영어만 인식한것만
def is_english(text):
    # 문자열이 None이 아닐 때 영어 문자가 포함되어 있는지 확인
    if isinstance(text, str):
        return bool(re.match(r'^[\x00-\x7F]*$', text))
    return False

# 영어가 아닌 행의 인덱스 찾기 (조건 반전)
indices_to_drop = Example1[~Example1['review_text'].apply(is_english)].index

# 기존 데이터프레임에서 영어가 아닌 행 삭제
Example1.drop(indices_to_drop, inplace=True)

# 결과 출력
Example1

,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
2,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,beautiful game,True,english,76561198016025460,206.0,1727243929
3,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,This game is so good and satisfying. I highly ...,True,english,76561198985925356,286.0,1727243851
4,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,"Medevil-fantasy vibes, The Game",True,english,76561199482488960,318.0,1727243723
5,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,"Extremely enjoyable and relaxing, soundtrack a...",True,english,76561198176194394,109.0,1727243579
6,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,you can pat the sheep and ducks !!!,True,english,76561198204216080,77.0,1727243166
...,...,...,...,...,...,...,...,...,...,...,...
362,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,Never stopped playing this game when I got it....,True,english,76561198079702081,NaN,1724909058
363,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,Good,True,english,76561198356910715,NaN,1724905629
364,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,LENA CHAN UUUUUUUOOOOOOOOOHHHHHHHHHH,True,english,76561198198292809,NaN,1724904791
365,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,DJMAX BOOBA UPDATE YES YES YES YES YES YES YES,True,english,76561197997052272,NaN,1724903183


In [26]:
#댓글 날짜, 닉네임, 사용자 플레이 시간 전처리

# 댓글 날짜 
Example1['playtime_at_review'].value_counts()
Example1['playtime_at_review'].isnull().sum() # 417
Example1['playtime_at_review'].value_counts().sum() # 6581
Example1['playtime_at_review']=Example1['playtime_at_review']/60
Example1['playtime_at_review']=Example1['playtime_at_review'].fillna(0)
Example1['playtime_at_review'].isnull().sum()
Example1['playtime_at_review'].astype(int)


# 사용자 플레이 시간 
Example1['timestamp_created'].isnull().sum() # 0
Example1['timestamp_created'].value_counts().sum() # 6581
#  Unix 타임스탬프
import pandas as pd

Example1['timestamp_created']
Example1['timestamp_created']=pd.to_datetime(Example1['timestamp_created'], unit='s')




In [27]:
Example1['game_name']

2                                             Tiny Glade
3                                             Tiny Glade
4                                             Tiny Glade
5                                             Tiny Glade
6                                             Tiny Glade
                             ...                        
362    DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET
363    DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET
364    DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET
365    DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET
366    DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET
Name: game_name, Length: 350, dtype: object

In [28]:
Example1.to_csv('Casual.csv')

In [29]:
ex=pd.read_csv('Casual.csv')
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          350 non-null    int64  
 1   game_name           350 non-null    object 
 2   game_genres         350 non-null    object 
 3   game_image          350 non-null    object 
 4   game_release_date   350 non-null    object 
 5   game_price          350 non-null    float64
 6   review_text         350 non-null    object 
 7   review_score        350 non-null    bool   
 8   language            350 non-null    object 
 9   author              350 non-null    int64  
 10  playtime_at_review  350 non-null    float64
 11  timestamp_created   350 non-null    object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 30.5+ KB


In [30]:

ex

,Unnamed: 0,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,2,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,beautiful game,True,english,76561198016025460,3.433333,2024-09-25 05:58:49
1,3,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,This game is so good and satisfying. I highly ...,True,english,76561198985925356,4.766667,2024-09-25 05:57:31
2,4,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,"Medevil-fantasy vibes, The Game",True,english,76561199482488960,5.300000,2024-09-25 05:55:23
3,5,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,"Extremely enjoyable and relaxing, soundtrack a...",True,english,76561198176194394,1.816667,2024-09-25 05:52:59
4,6,Tiny Glade,"Casual, Indie, Simulation",https://shared.akamai.steamstatic.com/store_it...,2024-09-23,14020.0,you can pat the sheep and ducks !!!,True,english,76561198204216080,1.283333,2024-09-25 05:46:06
...,...,...,...,...,...,...,...,...,...,...,...,...
345,362,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,Never stopped playing this game when I got it....,True,english,76561198079702081,0.000000,2024-08-29 05:24:18
346,363,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,Good,True,english,76561198356910715,0.000000,2024-08-29 04:27:09
347,364,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,LENA CHAN UUUUUUUOOOOOOOOOHHHHHHHHHH,True,english,76561198198292809,0.000000,2024-08-29 04:13:11
348,365,DJMAX RESPECT V - CLEAR PASS : S13 PREMIUM TICKET,"Action, Casual, Sports",https://shared.akamai.steamstatic.com/store_it...,2024-08-28,14800.0,DJMAX BOOBA UPDATE YES YES YES YES YES YES YES,True,english,76561197997052272,0.000000,2024-08-29 03:46:23
